# (實驗二) 根據天氣, 衣物穿搭建議程式


 
![alt text][logo]
[logo]: http://fit7.cn/UploadFiles/20154201617247.png "Logo Title Text 2"


## a. Configuring

In [1]:
from requests import get
from datetime import datetime, timedelta
from json import loads
from pprint import pprint
import time

APIKEY = "6872bed60bffc1bef999a2e3e98c4654" #apikey_openweathermap



## b. Get where user is and look up city id accordingly

In [2]:
def get_city_id_v3():
    city_id = False
    print('讀取城市地理資訊.......')
    with open('openweather_city_list.json',encoding='utf-8') as fd:
        data = [loads(line) for line in fd] #convert a JSON object to a dictionary, using the loads method
        print('地理資訊讀取完成!')
        city = input("你準備拜訪哪裡? 請輸入英文城市或地區名")
    for row in data:
        if row['name'] == city.title():
            answer = input('欲拜訪城市在 ' + row['country'] + '?' + '\n' + '輸入 y 或 yes 確認')
            if answer == 'y' or answer == 'yes' or answer == 'Yes': #OKAY syntax
                city_id = row['_id']
                break
            else:
                print('欲拜訪城市' +  + row['name'] + ' 不在 ' + row['country'])
    if city_id is False:
        print('這地理位址不存在!')      
    return city_id

## c. Get a 5-Day Forecast from openweathermap

In [3]:
def get_weather_data(city_id):
    access_url = 'http://api.openweathermap.org/data/2.5/forecast?id={0}&APPID={1}'.format(city_id,APIKEY)
    print("抓取氣象資料從 ", access_url)
    weather_data = get(access_url)
    print("抓取成功!")
    return weather_data.json()


## d.Get the arrival date and time 

In [4]:
# The date and time from the JSON file is in a specific format, YYYY-MM-DD HH:00:00
# ask user what day and hour they plan to arrive at the city

def get_arrival():
    today = datetime.now()
    margin = timedelta(days = 4)
    max_day = today + timedelta(days=4) #need to give them a range of dates to choose from.
    print('請問你幾月幾號到達目的地?')
    day = False
    while True:
        print("請選擇日期從 " + today.strftime('%d/%m/%y'), '到', max_day.strftime('%d/%m/%y'))
        date = input()
        date = datetime.strptime(date, "%d/%m/%y")
        if date.strftime('%d') == today.strftime('%d'):
            date += timedelta(hours=23)
        if today <= date <= max_day:
            break
        else:
            print("你沒有選擇對的日期, 請重新輸入!")
    print('何時到達?')
    print('0 - 24')
    hour = int(input())
    hour = hour - hour % 3 #The forecasts are only once every 3 hours, starting at 00:00:00.
    hour = datetime.strptime(str(hour), '%H')
    arrival = date.strftime('%Y-%m-%d') + ' ' + hour.strftime('%H') + ':00:00'
    print("你選擇到達日期與時間:", arrival)
    return arrival


## e. Get the forecase for the required date and time

In [5]:
def get_forecast(arrival, weather_data):
    forcast = False
    for item in weather_data['list']:
        if item['dt_txt'] == arrival:
            forcast = item
            return forcast
    if not forcast:
        print('對應天氣預報資料不存在!')

## f. parse the data in forecase to covert to readable format

In [6]:
def get_readable_forecast(forecast):
    weather = {}
    print(forecast['clouds']['all'])
    weather['cloudiness'] = forecast['clouds']['all']
    weather['temperature'] = float(forecast['main']['temp'])
    weather['humidity'] = int(forecast['main']['humidity'])
    weather['description'] = forecast['weather'][0]['description'] 
    weather['wind'] = float(forecast['wind']['speed'])
    try:
            #if '3h' in forecast['rain']:
            weather['rain'] = float(forecast['rain']['3h'])
    except:
        print('no rain data is available')
        weather['rain'] = 0.0
        pass
    if debug:
        print('------------------------------------DEBUG BEGIN--------------------------------------------------')
        print('debug: cloudiness is ' + str(weather['cloudiness']))
        print('debug: temperature is ' + str(weather['temperature']))
        print('debug: humidity is ' + str(weather['humidity']))
        print('debug: description is ' + weather['description'])
        print('debug: wind is ' + str(weather['wind']))
        print('debug: rain is ' + str(weather['rain']))
        print('------------------------------------DEBUG END--------------------------------------------------\n')
    return weather

## g. to determine what should wear

In [7]:
# 1. sunny by cloudiness --> a hat or sunglasses
# 2. no rain, light rain, moderate rain, heavy rain, and violent rain by degree of rain --> no umbrella, a hood or umbrella, umbrella, 
# an umbrella and a waterproof top and life-jacket
# 3. freezing, cold, not too cold and hot by temperature --> a heavy coat, a coat or thick jumper, light jumper, Shorts and T-shirt weather
# 4. wind, light breeze, calm by wind --> a jacket, long sleeves, nothing to worry

In [8]:
def get_clothes(weather):
    print('------------------------------------穿搭推薦結果--------------------------------------------------')
    print('大致天氣狀況為 {}'.format(weather['description']))
    # determine by cloudiness
    if weather['cloudiness'] < 10:
        print('天氣晴朗無雲, 也許需要帶帽子或太陽眼鏡')
    # determine by rain
    if weather['rain'] == 0 or weather['rain'] == 0.0 :
        print("當天不太可能下雨, 可以不用帶傘出門")
    elif weather['rain']/3 < 2.5:
        print("當天可能會下毛毛雨, 可以考慮帶傘或穿著帽T出門")
    elif weather['rain']/3 < 7.6:
        print("當天可能會下陣雨, 請帶傘出門")
    elif weather['rain']/3 < 50:
        print("當天可能會下大雨, 請帶傘與穿著防水衣物出門")
    elif weather['rain']/3 > 50:
        print("當天可能會有超級大雨, 也許考慮穿著救生衣")
    # determine by temperature
    if weather['temperature'] < 273:
        print("當天氣溫極低, 請穿著保暖厚重大衣")
    elif weather['temperature'] < 283:
        print("當天氣溫偏冷, 請穿著外套或保暖毛衣")
    elif weather['temperature'] < 293:
        print("當天氣溫沒有很冷, 也許可穿著輕便長袖")
    elif weather['temperature'] < 303:
        print("當天天氣嚴熱, 穿著短袖短褲")
    # determine by wind
    if weather['wind'] > 30:
        print("當天風大, 也許可穿著防風外套")
    elif weather['wind'] > 10:
        print("當天會起微微的風, 也許可穿著防風衣物")
    else:
        print("當天是個沒風的日子, 不需要穿著防風衣物")
    
    print('------------------------------------結束-----------------------------------------------------------')

## h. test and finish 

In [9]:
def main():
    global debug 
    debug = True
    city_id = get_city_id_v3()
    weather_data = get_weather_data(city_id)
    arrival = get_arrival()
    forecast = get_forecast(arrival, weather_data)
    weather = get_readable_forecast(forecast)
    get_clothes(weather)

main()

讀取城市地理資訊.......
地理資訊讀取完成!
你準備拜訪哪裡? 請輸入英文城市或地區名taiwan
欲拜訪城市在 TW?
輸入 y 或 yes 確認y
抓取氣象資料從  http://api.openweathermap.org/data/2.5/forecast?id=1668284&APPID=6872bed60bffc1bef999a2e3e98c4654
抓取成功!
請問你幾月幾號到達目的地?
請選擇日期從 08/06/16 到 12/06/16
09/06/16
何時到達?
0 - 24
18
你選擇到達日期與時間: 2016-06-09 18:00:00
92
------------------------------------DEBUG BEGIN--------------------------------------------------
debug: cloudiness is 92
debug: temperature is 290.934
debug: humidity is 96
debug: description is light rain
debug: wind is 0.42
debug: rain is 1.68
------------------------------------DEBUG END--------------------------------------------------

------------------------------------穿搭推薦結果--------------------------------------------------
大致天氣狀況為 light rain
當天可能會下毛毛雨, 可以考慮帶傘或穿著帽T出門
當天氣溫沒有很冷, 也許可穿著輕便長袖
當天是個沒風的日子, 不需要穿著防風衣物
------------------------------------結束-----------------------------------------------------------
